### Contraste des lésions / voisinages en 3D (01 -> 11)

In [ ]:
import SimpleITK as sitk
import numpy as np
import csv
from pathlib import Path

data_path = data_path = Path.cwd().parent.parent.parent / "data"

lesion_contrast_dir_path = data_path / "11_lesion_contrast_csv_dir"
if not lesion_contrast_dir_path.exists():
    lesion_contrast_dir_path.mkdir(parents = True, exist_ok = True)

In [ ]:
"""
01 -> 11
Ce script prend environ 9 minutes
"""

mslesseg_path = data_path / "01_MSLesSeg_Dataset"

# ------------------------------------------------------------
# 1. Paramètres
# ------------------------------------------------------------

kernel = sitk.sitkBall          # type de noyau pour dilatation
radius = (1, 1, 1)              # dilatation 1 voxel dans chaque direction

# ------------------------------------------------------------
# 2. Boucle sur tous les patients / masques
# ------------------------------------------------------------

for mask_path in mslesseg_path.rglob("*_MASK.nii.gz"):

    # Nom du patient + timepoint
    arg = mask_path.stem.split("_")
    if len(arg) == 3:
        patient_id, timepoint = arg[0], arg[1]
        name = f"{patient_id}_{timepoint}"
    elif len(arg) == 2:
        patient_id = arg[0]
        timepoint = "T1"
        name = patient_id

    print(f"\nTraitement {name}")

    # --------------------------------------------------------
    # 2.1 Identifier toutes les séquences disponibles
    # --------------------------------------------------------
    modalities = []
    for img_path in mask_path.parent.glob("*_*.nii.gz"):
        if "_MASK" not in img_path.stem:
            mod = (img_path.stem.split("_")[-1]).split(".")[0]  # T1, T2, FLAIR
            modalities.append((mod, img_path))

    if len(modalities) == 0:
        print(f"Aucune séquence trouvée pour {name}, skipping.")
        continue

    # --------------------------------------------------------
    # 2.2 Lecture du masque et extraction des labels
    # --------------------------------------------------------
    mask_img = sitk.Cast(sitk.ReadImage(mask_path) > 0, sitk.sitkUInt8)
    mask_labels = sitk.ConnectedComponent(mask_img)
    mask_labels = sitk.RelabelComponent(mask_labels)
    labels_array = sitk.GetArrayFromImage(mask_labels)
    num_lesions = int(labels_array.max())
    print(f"Nombre de lésions détectées : {num_lesions}")

    if num_lesions == 0:
        continue

    # --------------------------------------------------------
    # 2.3 Boucle sur toutes les séquences (T1, T2, FLAIR)
    # --------------------------------------------------------
    for modality, img_path in modalities:

        t_img = sitk.Cast(sitk.ReadImage(img_path), sitk.sitkFloat64)
        t_array = sitk.GetArrayFromImage(t_img)

        results = []

        for lesion_id in range(1, num_lesions + 1):
            lesion_mask_bool = (labels_array == lesion_id)
            lesion_mask_sitk = sitk.GetImageFromArray(lesion_mask_bool.astype(np.uint8))
            lesion_mask_sitk.CopyInformation(t_img)

            # Moyenne lésion
            lesion_voxels = t_array[labels_array == lesion_id]
            lesion_mean = round(lesion_voxels.mean(), 1)

            # Dilatation pour le voisinage
            dilated_mask = sitk.BinaryDilate(lesion_mask_sitk, radius, kernelType=kernel)
            dilated_array = sitk.GetArrayFromImage(dilated_mask)

            neighborhood_mask = (dilated_array > 0) & (labels_array != lesion_id)
            neighborhood_voxels = t_array[neighborhood_mask]

            if neighborhood_voxels.size > 0:
                neighborhood_mean = round(neighborhood_voxels.mean(), 1)
                contrast = round(lesion_mean / neighborhood_mean, 2)
            else:
                neighborhood_mean = ""
                contrast = ""

            results.append({
                "patient": patient_id,
                "timepoint": timepoint,
                "modality": modality,
                "lesion_id": lesion_id,
                "lesion_mean": lesion_mean,
                "neighborhood_mean": neighborhood_mean,
                "contrast_lesion_neighborhood": contrast
            })

        # --------------------------------------------------------
        # 2.4 Sauvegarde CSV pour chaque modalité
        # --------------------------------------------------------
        csv_file = lesion_contrast_dir_path / f"{name}_{modality}_lesion_contrast.csv"
        keys = ["patient","timepoint","modality","lesion_id","lesion_mean","neighborhood_mean","contrast_lesion_neighborhood"]

        with open(csv_file, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            for row in results:
                writer.writerow(row)

        print(f"CSV sauvegardé : {csv_file.name}")